A copy from https://github.com/Heumi/BEGAN-tensorflow/tree/master/src/function

To test the difference

http://www.foldl.me/uploads/2015/conditional-gans-face-generation/paper.pdf

https://github.com/nightrome/really-awesome-gan

https://arxiv.org/pdf/1705.09966.pdf

https://arxiv.org/pdf/1703.05192.pdf

https://arxiv.org/pdf/1606.03657.pdf

https://arxiv.org/pdf/1704.02166.pdf

https://arxiv.org/pdf/1709.03842.pdf

In [1]:
import os
import scipy.misc as scm
import numpy as np

def make_project_dir(project_dir):
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
        os.makedirs(os.path.join(project_dir, 'models'))
        os.makedirs(os.path.join(project_dir, 'result'))
        os.makedirs(os.path.join(project_dir, 'result_test'))


def get_image(img_path):
    img = scm.imread(img_path)/255. 
    img = img[..., ::-1]  # rgb to bgr
    return img

def get_label(path, size):
    label = int(path[-5])
    one_hot = np.zeros(size)
    one_hot[ label ] = 0.9
    one_hot[ one_hot==0 ] = 0.1
    return one_hot

def inverse_image(img):
    img = img * 255.
    img[img > 255] = 255
    img[img < 0] = 0
    img = img[..., ::-1] # bgr to rgb
    return img

def pair_expressions(paths):
    subject_exprs = []
    subject_pairs = []
    all_pairs = []
    last_subject = 0

    # Pair all expression of a subject
    for path in paths:
        subject = int(path[-9:-6])

        if subject != last_subject and last_subject != 0:
            subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
            all_pairs.extend(subject_pairs)
            subject_exprs = []

        subject_exprs.append(path)
        last_subject = subject

    # Last subject
    subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
    all_pairs.extend(subject_pairs)
    return all_pairs

def get_shape_c(tensor): # static shape
    return tensor.get_shape().as_list()


In [2]:
import tensorflow as tf
import numpy as np

dropout = tf.layers.dropout
bn = tf.layers.batch_normalization

def conv(x, filter_shape, bias=False, stride=1, padding="VALID", name="conv2d"):
    kw, kh, nin, nout = filter_shape

    initializer = tf.random_normal_initializer(0., 0.01)
    with tf.variable_scope(name):
        x = tf.layers.conv2d(x, filters=nout, kernel_size=(kw, kh), strides=(stride, stride), padding=padding, 
                             use_bias=bias, kernel_initializer=initializer)
    return x

def deconv(x, filter_shape, bias=False, stride=1, padding="VALID", name="conv2d_transpose"):
    kw, kh, nin, nout = filter_shape

    initializer = tf.random_normal_initializer(0., 0.01)
    with tf.variable_scope(name):
        x = tf.layers.conv2d_transpose(x, filters=nout, kernel_size=(kw, kh), strides=(stride, stride), padding=padding, 
                                       use_bias=bias, kernel_initializer=initializer)
    return x


def fc(x, output_shape, bias=True, name='fc'):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    initializer = tf.random_normal_initializer(0., 0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        if bias:
            b = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x

def pool(x, r=2, s=1):
    return tf.nn.avg_pool(x, ksize=[1, r, r, 1], strides=[1, s, s, 1], padding="SAME")


def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))


def resize_nn(x, size):
    return tf.image.resize_nearest_neighbor(x, size=(int(size), int(size)))

def add_nontied_bias(x, initializer=None):
    with tf.variable_scope('add_nontied_bias'):
        if initializer is not None:
            bias = tf.get_variable('bias', shape=x.get_shape().as_list()[1:], trainable=True, initializer=initializer)
        else:
            bias = tf.get_variable('bias', shape=x.get_shape().as_list()[1:], trainable=True, initializer=tf.zeros_initializer())

        output = x + bias
    return output


In [3]:
import glob
import time
import numpy as np
import tensorflow as tf

class op_base:
    def __init__(self, sess, project_name):
        self.sess = sess

        # Train
        self.flag = True #args.flag
        self.gpu_number = 0 #args.gpu_number
        self.project = project_name #"test_began" #args.project

        # Train Data
        self.data_dir = "./Face_data/Faces_with_expression_label/dataset_64x64" #args.data_dir #./Data
        self.dataset = "expr" #args.dataset  # celeba
        self.data_size = 64 #args.data_size  # 64 or 128
        self.data_opt = "crop" #args.data_opt  # raw or crop
        self.data_label_vector_size = 7 #size of one-hot-encoded label vector

        # Train Iteration
        self.niter = 5000 #50 #args.niter
        self.niter_snapshot = 500 #args.nsnapshot
        self.max_to_keep = 5 #args.max_to_keep

        # Train Parameter
        self.batch_size = 32#16 #args.batch_size
        self.learning_rate = 1e-4 #args.learning_rate
        self.mm = 0.5 #args.momentum
        self.mm2 = 0.999 #args.momentum2
        self.lamda = 0.001 #args.lamda
        self.gamma = 0.5 #args.gamma
        self.filter_number = 64 #args.filter_number
        self.input_size = 64 #args.input_size
        self.embedding = 512 #64 #args.embedding

        # Result Dir & File
        self.project_dir = 'assets/{0}_{1}_{2}_{3}/'.format(self.project, self.dataset, self.data_opt, self.data_size)
        self.ckpt_dir = os.path.join(self.project_dir, 'models')
        self.model_name = "{0}.model".format(self.project)
        self.ckpt_model_name = os.path.join(self.ckpt_dir, self.model_name)

        # etc.
        if not os.path.exists('assets'):
            os.makedirs('assets')
        make_project_dir(self.project_dir)

    def load(self, sess, saver, ckpt_dir):
        ckpt = tf.train.get_checkpoint_state(ckpt_dir)
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(ckpt_dir, ckpt_name))

In [4]:
import glob
import time
import datetime


class Operator(op_base):
    def __init__(self, sess, project_name):
        op_base.__init__(self, sess, project_name)
        self.build_model()

    def build_model(self):
        # Input placeholder
        self.x1 = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='x1')
        self.x2 = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='x2')
        self.c1 = tf.placeholder(tf.float32, shape=[None, self.data_label_vector_size], name='c1')
        self.c2 = tf.placeholder(tf.float32, shape=[None, self.data_label_vector_size], name='c2')
        self.lr = tf.placeholder(tf.float32, name='lr')
        self.train_g = tf.placeholder(tf.bool, [])
        self.train_d = tf.placeholder(tf.bool, [])

        self.G_l = self.encoder(self.x2, self.c2, train=self.train_g, reuse=False)
        self.l1 = self.encoder(self.x1, self.c1, train=False, reuse=True)
        self.G_i = self.decoder(self.l1, self.c1, train=self.train_g, reuse=False)
        
        self.D_G_i, self.D_G_i_logits = self.discriminator(self.G_i, self.l1, self.c1, train=self.train_d, reuse=False)
        self.D_G_l, self.D_G_l_logits = self.discriminator(self.x2, self.G_l, self.c2, train=self.train_d, reuse=True)

        # Generate test image
        self.l_test = self.encoder(self.x1, self.c1, train=False, reuse=True)
        self.G_test = self.decoder(self.l_test, self.c2, train=False, reuse=True)

        # Loss
        self.d_loss = tf.reduce_mean(tf.nn.softplus(-self.D_G_l_logits) + tf.nn.softplus(self.D_G_i_logits))
        self.g_loss = tf.reduce_mean(tf.nn.softplus(self.D_G_l_logits) + tf.nn.softplus(-self.D_G_i_logits))

        # Variables
        encoder_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "encoder_")
        decoder_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "decoder_")
        discriminator_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "discriminator_")

        # Optimizer
        # update parameters for BN first
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            self.opt_d = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.d_loss, var_list=discriminator_vars)
            self.opt_g = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.g_loss, var_list=encoder_vars+decoder_vars)


        # initializer
        self.sess.run(tf.global_variables_initializer())

        # tf saver
        self.saver = tf.train.Saver(max_to_keep=(self.max_to_keep))

        try:
            self.load(self.sess, self.saver, self.ckpt_dir)
        except:
            # save full graph
            self.saver.save(self.sess, self.ckpt_model_name, write_meta_graph=True)

        # Summary
        if self.flag:
            tf.summary.scalar('loss/d_loss', self.d_loss)
            tf.summary.scalar('loss/g_loss', self.g_loss)
            #tf.summary.scalar('loss/test_loss', self.test_loss)
            self.merged = tf.summary.merge_all()
            self.writer = tf.summary.FileWriter(self.project_dir, self.sess.graph)

    def train(self, train_flag):
        # load data
        data_path = self.data_dir
        #test_data_path = "./Face_data/Faces_with_expression_label/dataset_64x64" # expressions

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            np.save(data_path + '.npy', data)

        print('Shuffle ....')
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        print('Shuffle Done')

        # initial parameter
        start_time = time.time()
        lr = np.float32(self.learning_rate)
        self.count = 0

        for epoch in range(self.niter):
            batch_idxs = len(data) // self.batch_size

            for idx in range(0, batch_idxs):
                self.count += 1

                batch_files = data[idx * self.batch_size: (idx + 1) * self.batch_size]
                batch_x1 = [get_image(batch_file[0]) for batch_file in batch_files]
                batch_x2 = [get_image(batch_file[1]) for batch_file in batch_files]
                batch_c1 = [get_label(batch_file[0], self.data_label_vector_size) for batch_file in batch_files]
                batch_c2 = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]

                # opt & feed list 
                opts = [self.opt_d, self.opt_g, self.d_loss, self.g_loss, self.merged]
                feed_dict = {self.x1: batch_x1, self.x2: batch_x2, 
                             self.c1: batch_c1, self.c2: batch_c2,
                             self.lr: lr, self.train_g: True, self.train_d: True}
          
                # run tensorflow
                _, _, d_loss, g_loss, summary = self.sess.run(opts, feed_dict=feed_dict)
                
                if self.count % 2 == 1:
                    print("Epoch: [%2d] [%4d/%4d] time: %4.4f, "
                          "d_loss: %.6f, g_loss: %.6f"
                          % (epoch, idx, batch_idxs, time.time() - start_time,
                             d_loss, g_loss))

                # write train summary
                self.writer.add_summary(summary, self.count)

                # Test during Training
                if self.count % self.niter_snapshot == (self.niter_snapshot - 1):
                    # save & test
                    self.saver.save(self.sess, self.ckpt_model_name, global_step=self.count, write_meta_graph=False)
                    self.test_expr(train_flag)
                    #self.test_celebra(train_flag)

    def test_celebra(self, train_flag=True):
        print('Test Sample Generation...')
        # generate output
        img_num = 8*8
        output_f = int(np.sqrt(img_num))
        in_img_num = output_f
        img_size = self.data_size
        gen_img_num = img_num - output_f
        label_size = self.data_label_vector_size
        
        # load data test
        data_path = "./Face_data/Celeba/dataset_64x64" # expressions

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            np.save(data_path + '.npy', data)

        # shuffle test data
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]

        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])

        test_files = data[0: output_f]
        test_data = [get_image(test_file) for test_file in test_files]
        test_data = np.repeat(test_data, [label_size]*in_img_num, axis=0)
        test_data_o = [scm.imread(test_file) for test_file in test_files]
        
        # get one-hot labels
        int_labels = list(range(label_size))
        one_hot = np.zeros((label_size, label_size))
        one_hot[np.arange(label_size), int_labels] = 1
        target_labels = np.tile(one_hot, (output_f, 1))
        
        
        output_gen = (self.sess.run(self.ae_test, feed_dict={self.x: test_data, 
                                                             self.c: target_labels}))  # generator output

        output_gen = [inverse_image(output_gen[i]) for i in range(gen_img_num)]

        for i in range(output_f):
            for j in range(output_f):
                if j == 0:
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_data_o[i]
                else:
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = output_gen[(j-1) + (i * int(output_f-1))]

        # output save
        scm.imsave(self.project_dir + '/result/' + str(self.count) + '_celebra_output.bmp', im_output_gen)
        
    def test_expr(self, train_flag=True):
        print('Test Sample Generation...')
        # generate output
        img_num =  36 #self.batch_size
        display_img_num = int(img_num / 3)
        img_size = self.data_size

        output_f = int(np.sqrt(img_num))
        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])
        
        # load data
        data_path = self.data_dir

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            data = pair_expressions(data)
            np.save(data_path + '.npy', data)

        # Test data shuffle
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        
        batch_files = data[0: display_img_num]
        test_inputs = [get_image(batch_file[0]) for batch_file in batch_files]
        test_inputs_o = [scm.imread((batch_file[0])) for batch_file in batch_files]
        test_targets = [scm.imread((batch_file[1])) for batch_file in batch_files]
        test_inputs_labels = [get_label(batch_file[0], self.data_label_vector_size) for batch_file in batch_files]
        test_target_labels = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]

        feed_dict={self.x1: test_inputs, self.c1: test_inputs_labels, self.c2: test_target_labels}
        output_gen = (self.sess.run(self.G_test, feed_dict=feed_dict))  # generator output

        output_gen = [inverse_image(output_gen[i]) for i in range(display_img_num)]

        for i in range(output_f): # row
            for j in range(output_f): # col
                if j % 3 == 0: # input img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_inputs_o[int(j / 3) + (i * int(output_f / 3))]
                elif j % 3 == 1: # output img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = output_gen[int(j / 3) + (i * int(output_f / 3))]
                else: # target img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_targets[int(j / 3) + (i * int(output_f / 3))]
                   

        labels = np.argmax(test_target_labels, axis=1)
        label_string = ''.join(str(int(l)) for l in labels)
        # output save
        scm.imsave(self.project_dir + '/result/' + str(self.count) + '_' + label_string 
                   + '_output.bmp', im_output_gen)
        

In [5]:

class ALI(Operator):
    def __init__(self, sess, project_name):
        Operator.__init__(self, sess, project_name)


    def encoder(self, x, c, reuse=None, train=True):
        with tf.variable_scope('encoder_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            c_num = self.data_label_vector_size
            h = self.embedding
            p = "VALID"

            with tf.variable_scope('block1'):
                x = conv(x, [2, 2, 3, f], stride=1, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)

                # Add classes as channels to the outcome of first layer as described in IcGAN paper
                x = tf.concat([x, tf.tile(tf.reshape(c, [-1, 1, 1, get_shape_c(c)[-1]]),\
                                          [1, x.get_shape().as_list()[1], x.get_shape().as_list()[2], 1])],\
                                          axis=3)
            
            with tf.variable_scope('block2'):
                x = conv(x, [7, 7, f+c_num, f*2], stride=2, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block3'):
                x = conv(x, [5, 5, f*2, f*4], stride=2, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block4'):
                x = conv(x, [7, 7, f*4, f*4], stride=2, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block5'):
                x = conv(x, [4, 4, f*8, f*8], stride=1, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block6'):
                x = conv(x, [1, 1, f*8, h], stride=1, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block7_mu'):
                x_mu = conv(x, [1, 1, h, h], stride=1, padding=p)
                x_mu = add_nontied_bias(x_mu)

            with tf.variable_scope('block7_sigma'):
                x_sigma = conv(x, [1, 1, h, h], stride=1, padding=p)
                x_sigma = add_nontied_bias(x_sigma)
                x_sigma = tf.exp(x_sigma)
                
            rng = tf.random_normal(shape=tf.shape(x_mu))
            x = (rng * x_sigma) + x_mu

            #x = fc(x, h, name='fc')
        return x

    def decoder(self, x, c, reuse=None, train=True):
        with tf.variable_scope('decoder_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            c_num = self.data_label_vector_size
            h = self.embedding
            p = "VALID"
            
            c = tf.reshape(c, [-1, 1, 1, c_num])
            x = tf.reshape(x, [-1, 1, 1, h])
            x = tf.concat([x, c], axis=3)

            with tf.variable_scope('block1'):
                x = deconv(x, [4, 4, h+c_num, f*8], stride=1, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block2'):
                x = deconv(x, [7, 7, f*8, f*4], stride=2, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
            
            with tf.variable_scope('block3'):
                x = deconv(x, [5, 5, f*4, f*4], stride=2, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block4'):
                x = deconv(x, [7, 7, f*4, f*2], stride=2, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block5'):
                x = deconv(x, [2, 2, f*2, f], stride=1, padding=p)
                x = bn(x, axis=3, center=True, scale=True, training=train)
                x = tf.nn.elu(x)
                
            with tf.variable_scope('block6'):
                x = conv(x, [1, 1, f, 3], stride=1, bias=True, padding=p)
                x = tf.nn.sigmoid(x)

        return x
    
    def discriminator(self, im, latent, c, reuse=None, train=True):
        with tf.variable_scope('discriminator_') as scope:
            if reuse:
                scope.reuse_variables()
            
            d_i = im
            d_l = latent
            f = self.filter_number
            c_num = self.data_label_vector_size
            h = self.embedding
            p = "VALID"
            
            with tf.variable_scope('i_'):
                
                with tf.variable_scope('block1'):
                    d_i = conv(d_i, [2, 2, 3, f], stride=1, bias=True, padding=p)
                    d_i = bn(d_i, axis=3, center=True, scale=True, training=train)
                    d_i = tf.nn.elu(d_i)
                    
                with tf.variable_scope('block2'):
                    d_i = conv(d_i, [7, 7, f, f*2], stride=2, bias=True, padding=p)
                    d_i = bn(d_i, axis=3, center=True, scale=True, training=train)
                    d_i = tf.nn.elu(d_i)
                
                with tf.variable_scope('block3'):
                    d_i = conv(d_i, [5, 5, f*2, f*4], stride=2, padding=p)
                    d_i = bn(d_i, axis=3, center=True, scale=True, training=train)
                    d_i = tf.nn.elu(d_i)
                
                with tf.variable_scope('block4'):
                    d_i = conv(d_i, [7, 7, f*4, f*4], stride=2, bias=True, padding=p)
                    d_i = bn(d_i, axis=3, center=True, scale=True, training=train)
                    d_i = tf.nn.elu(d_i)
                
                with tf.variable_scope('block5'):
                    d_i = conv(d_i, [4, 4, f*4, f*8], stride=1, bias=True, padding=p)
                    d_i = bn(d_i, axis=3, center=True, scale=True, training=train)
                    d_i = tf.nn.elu(d_i)
                
            with tf.variable_scope('l_'):
                
                d_l = tf.reshape(d_l, [-1, 1, 1, h])
                
                with tf.variable_scope('block1'):    
                    d_l = dropout(d_l, rate=0.2, training=train)
                    d_l = conv(d_l, [1, 1, h, f*16], bias=False, stride=1, padding=p)
                    d_l = tf.nn.elu(d_l)
                    
                with tf.variable_scope('block2'):
                    d_l = dropout(d_l, rate=0.2, training=train)
                    d_l = conv(d_l, [1, 1, f*16, f*16], bias=False, stride=1, padding=p)
                    d_l = tf.nn.elu(d_l)
                    
            with tf.variable_scope('ilc_'):
                
                c = tf.reshape(c, [-1, 1, 1, c_num])
                d_ilc = tf.concat([d_i, d_l, c], axis=3)
                
                with tf.variable_scope('block1'):    
                    d_ilc = dropout(d_ilc, rate=0.2, training=train)
                    d_ilc = conv(d_ilc, [1, 1, f*8+f*16+c_num, f*32], stride=1, bias=True, padding=p)
                    d_ilc = tf.nn.elu(d_ilc)
                    
                with tf.variable_scope('block2'):    
                    d_ilc = dropout(d_ilc, rate=0.2, training=train)
                    d_ilc = conv(d_ilc, [1, 1, f*32, f*32], stride=1, bias=True, padding=p)
                    d_ilc = tf.nn.elu(d_ilc)
                    
                with tf.variable_scope('block3'):    
                    d_ilc = dropout(d_ilc, rate=0.2, training=train)
                    d_ilc = conv(d_ilc, [1, 1, f*32, 1], stride=1, bias=True, padding=p)
            
            logits = d_ilc
            sig = tf.nn.sigmoid(d_ilc)
            
            return sig, logits
                
                
                
            

In [6]:
import distutils.util
import os
import tensorflow as tf

''' config settings '''

project_name = "ConditionalALI_Face_2"
train_flag = True

'''-----------------'''

gpu_number = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #args.gpu_number

with tf.device('/gpu:{0}'.format(gpu_number)):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.90)
    config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)

    with tf.Session(config=config) as sess:
        model = ALI(sess, project_name)

        # TRAIN / TEST
        if train_flag:
            model.train(train_flag)
        else:
            model.test(train_flag)

Shuffle ....
Shuffle Done
Epoch: [ 0] [   0/ 316] time: 2.2000, d_loss: 1.383710, g_loss: 1.390110
Epoch: [ 0] [   2/ 316] time: 2.8546, d_loss: 1.375297, g_loss: 1.400406
Epoch: [ 0] [   4/ 316] time: 3.5078, d_loss: 1.384886, g_loss: 1.390762
Epoch: [ 0] [   6/ 316] time: 4.1624, d_loss: 1.363120, g_loss: 1.412010
Epoch: [ 0] [   8/ 316] time: 4.8206, d_loss: 1.352259, g_loss: 1.430207
Epoch: [ 0] [  10/ 316] time: 5.4786, d_loss: 1.310557, g_loss: 1.476757
Epoch: [ 0] [  12/ 316] time: 6.1384, d_loss: 1.257354, g_loss: 1.537143
Epoch: [ 0] [  14/ 316] time: 6.7927, d_loss: 1.196308, g_loss: 1.635256
Epoch: [ 0] [  16/ 316] time: 7.4541, d_loss: 1.118630, g_loss: 1.781234
Epoch: [ 0] [  18/ 316] time: 8.1126, d_loss: 0.983534, g_loss: 1.971413
Epoch: [ 0] [  20/ 316] time: 8.7730, d_loss: 0.871104, g_loss: 2.346608
Epoch: [ 0] [  22/ 316] time: 9.4296, d_loss: 0.947334, g_loss: 2.435038
Epoch: [ 0] [  24/ 316] time: 10.0847, d_loss: 0.794357, g_loss: 2.689817
Epoch: [ 0] [  26/ 316] 

Epoch: [ 0] [ 222/ 316] time: 75.6343, d_loss: 1.464056, g_loss: 2.981787
Epoch: [ 0] [ 224/ 316] time: 76.2996, d_loss: 0.567291, g_loss: 3.826739
Epoch: [ 0] [ 226/ 316] time: 76.9644, d_loss: 0.981439, g_loss: 2.910825
Epoch: [ 0] [ 228/ 316] time: 77.6287, d_loss: 0.697645, g_loss: 3.927514
Epoch: [ 0] [ 230/ 316] time: 78.2927, d_loss: 0.869495, g_loss: 2.872766
Epoch: [ 0] [ 232/ 316] time: 78.9567, d_loss: 0.887950, g_loss: 3.397305
Epoch: [ 0] [ 234/ 316] time: 79.6214, d_loss: 0.486024, g_loss: 3.700614
Epoch: [ 0] [ 236/ 316] time: 80.2872, d_loss: 0.521652, g_loss: 3.884615
Epoch: [ 0] [ 238/ 316] time: 80.9558, d_loss: 1.720816, g_loss: 4.236128
Epoch: [ 0] [ 240/ 316] time: 81.6228, d_loss: 0.446671, g_loss: 4.202503
Epoch: [ 0] [ 242/ 316] time: 82.2880, d_loss: 0.975312, g_loss: 3.196357
Epoch: [ 0] [ 244/ 316] time: 82.9521, d_loss: 0.528157, g_loss: 4.325714
Epoch: [ 0] [ 246/ 316] time: 83.6182, d_loss: 0.681957, g_loss: 3.437139
Epoch: [ 0] [ 248/ 316] time: 84.2829,

Epoch: [ 1] [ 126/ 316] time: 149.3035, d_loss: 0.216928, g_loss: 6.122698
Epoch: [ 1] [ 128/ 316] time: 149.9857, d_loss: 0.464081, g_loss: 6.019615
Epoch: [ 1] [ 130/ 316] time: 150.6678, d_loss: 0.470597, g_loss: 5.351743
Epoch: [ 1] [ 132/ 316] time: 151.3510, d_loss: 2.530237, g_loss: 3.989373
Epoch: [ 1] [ 134/ 316] time: 152.0324, d_loss: 0.591420, g_loss: 8.089653
Epoch: [ 1] [ 136/ 316] time: 152.7151, d_loss: 0.189740, g_loss: 8.211820
Epoch: [ 1] [ 138/ 316] time: 153.4016, d_loss: 0.186318, g_loss: 7.651402
Epoch: [ 1] [ 140/ 316] time: 154.0888, d_loss: 0.482397, g_loss: 6.021285
Epoch: [ 1] [ 142/ 316] time: 154.7747, d_loss: 0.158133, g_loss: 7.013131
Epoch: [ 1] [ 144/ 316] time: 155.4591, d_loss: 0.259846, g_loss: 5.722879
Epoch: [ 1] [ 146/ 316] time: 156.1417, d_loss: 0.759027, g_loss: 4.404072
Epoch: [ 1] [ 148/ 316] time: 156.8231, d_loss: 2.080277, g_loss: 3.481394
Epoch: [ 1] [ 150/ 316] time: 157.5017, d_loss: 0.447896, g_loss: 4.646549
Epoch: [ 1] [ 152/ 316] t

Epoch: [ 2] [  34/ 316] time: 225.9044, d_loss: nan, g_loss: nan
Epoch: [ 2] [  36/ 316] time: 226.5629, d_loss: nan, g_loss: nan
Epoch: [ 2] [  38/ 316] time: 227.2200, d_loss: nan, g_loss: nan
Epoch: [ 2] [  40/ 316] time: 227.8775, d_loss: nan, g_loss: nan
Epoch: [ 2] [  42/ 316] time: 228.5341, d_loss: nan, g_loss: nan


KeyboardInterrupt: 